In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/mnt/software/'
import sys
import gc
# assuming data, models, engine in flicc directory:
flicc_path = os.path.realpath("__file__").split('grid_search')[0]
sys.path.append(flicc_path)
import torch
from data import ClimateDataset
from models import ClassificationModel
from engine import Engine

import warnings
warnings.filterwarnings("ignore")

In [2]:
model_checkpoint='microsoft/deberta-v2-xlarge'

In [3]:
results = {'test_acc':[],
           'test_f1':[],
           'eval_acc':[],
           'eval_f1':[],
           'lr':[]}

In [4]:
r = 8
a = 8
lora_dropout = 0.0

learning_rates = [1.0e-6, 1.0e-5, 5.0e-5 ,1.0e-4, 1.0e-3]

for lr in learning_rates:
    print(f'Grid search {model_checkpoint}, learning rate {lr}')
    data = ClimateDataset(model_to_train=2,model_checkpoint=model_checkpoint,dataset_url=flicc_path,batch_size=8)
    data.setup_dataloaders()
    model = ClassificationModel(model_checkpoint=data.model_checkpoint,
                                num_labels=data.num_labels,
                                quantized_model=True,
                                lora=True,
                                r=r,
                                alpha=a,
                                dropout=lora_dropout)
    trainer = Engine(epochs=30,labels=data.labels)
    trainer.model = model.model
    trainer.run(lr=lr,
                wd=0.0,
                train_dataloader=data.train_dataloader,
                eval_dataloader=data.eval_dataloader,
                test_dataloader=data.test_dataloader,
                accumulation_steps=4,
                early_stop=3,
                is_quantized=True)
    with torch.no_grad():
        torch.cuda.empty_cache()
    del data, model, trainer
    gc.collect()

Grid search microsoft/deberta-v2-xlarge, learning rate 1e-06


Filter:   0%|          | 0/1796 [00:00<?, ? examples/s]

Filter:   0%|          | 0/256 [00:00<?, ? examples/s]

Filter:   0%|          | 0/457 [00:00<?, ? examples/s]

Map:   0%|          | 0/472 [00:00<?, ? examples/s]

...

1 / 30: Train Loss:	0.2773	Val Loss:	1.1024	Accuracy:	0.3866	F1:	0.2964	Test Accuracy:	0.4925	Test F1:	0.3855 *
2 / 30: Train Loss:	0.2738	Val Loss:	1.1011	Accuracy:	0.3697	F1:	0.1800	Test Accuracy:	0.3731	Test F1:	0.2256
3 / 30: Train Loss:	0.2754	Val Loss:	1.0993	Accuracy:	0.3782	F1:	0.1829	Test Accuracy:	0.3731	Test F1:	0.2064
4 / 30: Train Loss:	0.2720	Val Loss:	1.0969	Accuracy:	0.3782	F1:	0.1829	Test Accuracy:	0.3731	Test F1:	0.2056
No improvement for 3 epochs. Stopping early.
Grid search microsoft/deberta-v2-xlarge, learning rate 0.0001


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v2-xlarge and are newly initialized: ['pooler.dense.weight', 'classifier.weight', 'pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	0.2775	Val Loss:	1.1003	Accuracy:	0.3782	F1:	0.1829	Test Accuracy:	0.3881	Test F1:	0.2130 *
2 / 30: Train Loss:	0.2731	Val Loss:	1.0978	Accuracy:	0.3782	F1:	0.1829	Test Accuracy:	0.3881	Test F1:	0.2121
3 / 30: Train Loss:	0.2749	Val Loss:	1.0974	Accuracy:	0.3613	F1:	0.1924	Test Accuracy:	0.3731	Test F1:	0.2056 *
4 / 30: Train Loss:	0.2712	Val Loss:	1.0935	Accuracy:	0.3782	F1:	0.2757	Test Accuracy:	0.4627	Test F1:	0.3654 *
5 / 30: Train Loss:	0.2738	Val Loss:	1.0878	Accuracy:	0.3697	F1:	0.2603	Test Accuracy:	0.4179	Test F1:	0.3210
6 / 30: Train Loss:	0.2721	Val Loss:	1.0824	Accuracy:	0.3782	F1:	0.1977	Test Accuracy:	0.4478	Test F1:	0.3262
7 / 30: Train Loss:	0.2665	Val Loss:	1.0754	Accuracy:	0.3866	F1:	0.2011	Test Accuracy:	0.4328	Test F1:	0.3117
No improvement for 3 epochs. Stopping early.
Grid search microsoft/deberta-v2-xlarge, learning rate 0.001


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v2-xlarge and are newly initialized: ['pooler.dense.weight', 'classifier.weight', 'pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	0.2786	Val Loss:	1.1025	Accuracy:	0.3697	F1:	0.3187	Test Accuracy:	0.4179	Test F1:	0.3677 *
2 / 30: Train Loss:	0.2631	Val Loss:	1.0392	Accuracy:	0.4202	F1:	0.3845	Test Accuracy:	0.4627	Test F1:	0.4556 *
3 / 30: Train Loss:	0.2383	Val Loss:	0.8042	Accuracy:	0.5966	F1:	0.5995	Test Accuracy:	0.5970	Test F1:	0.5969 *
4 / 30: Train Loss:	0.1853	Val Loss:	0.6643	Accuracy:	0.7395	F1:	0.7220	Test Accuracy:	0.6418	Test F1:	0.6479 *
5 / 30: Train Loss:	0.1258	Val Loss:	0.6134	Accuracy:	0.7563	F1:	0.7502	Test Accuracy:	0.7612	Test F1:	0.7650 *
6 / 30: Train Loss:	0.0859	Val Loss:	0.5710	Accuracy:	0.8067	F1:	0.7990	Test Accuracy:	0.7164	Test F1:	0.7158 *
7 / 30: Train Loss:	0.0635	Val Loss:	0.5426	Accuracy:	0.8067	F1:	0.8088	Test Accuracy:	0.7313	Test F1:	0.7325 *
8 / 30: Train Loss:	0.0441	Val Loss:	0.5769	Accuracy:	0.8319	F1:	0.8333	Test Accuracy:	0.7910	Test F1:	0.7890 *
9 / 30: Train Loss:	0.0230	Val Loss:	0.6279	Accuracy:	0.8403	F1:	0.8411	Test Accuracy:	0.8209	Test F1:	0

In [5]:
%reset -f